In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import argparse
from tqdm import tqdm
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import tensorflow as tf

from autolab_core import YamlConfig

from sd_maskrcnn import utils
from sd_maskrcnn.config import MaskConfig
from sd_maskrcnn.dataset import ImageDataset
from sd_maskrcnn.coco_benchmark import coco_benchmark

from mrcnn import model as modellib, utils as utilslib, visualize

In [ ]:
config = YamlConfig('../cfg/benchmark.yaml')
inference_config = MaskConfig(config['model']['settings'])
inference_config.GPU_COUNT = 1
inference_config.IMAGES_PER_GPU = 1

In [ ]:
with tf.device('/gpu:0'):
    model = modellib.MaskRCNN(mode=config['model']['mode'], 
                          config=inference_config, 
                          model_dir=os.path.split(config['model']['path'])[0])

In [ ]:
import matplotlib.patches as patches

input_image = io.imread('/nfs/diskstation/projects/dex-net/segmentation/physical_experiments/single_obj_images/phoxi/images/climbing_hold_000007.png')
input_image, _, _, _, _ = utilslib.resize_image(input_image, 512, 512)
plt.imshow(input_image)
plt.show()

mask = np.sum(input_image, axis=2) != 0
mask = mask.reshape((mask.shape[0], mask.shape[1], 1))
bb = utilslib.extract_bboxes(mask)

bb = bb[0] 
print(bb)
float_bb = [bb[0] / mask.shape[0], bb[1] / mask.shape[1], bb[2] / mask.shape[0], 
     bb[3] / mask.shape[1]]

fix, ax = plt.subplots(1)

plt.imshow(np.sum(input_image, axis=2) != 0)
ax.add_patch(patches.Rectangle((bb[1], bb[0]), bb[3] - bb[1], bb[2] - bb[0],
                               facecolor='none', edgecolor='r')) 
plt.show()

print(float_bb)

plt.imshow(input_image[bb[0]:bb[2], bb[1]:bb[3]])
target=input_image[bb[0]:bb[2], bb[1]:bb[3]]

In [ ]:
filepath='/nfs/diskstation/projects/dex-net/segmentation/models/mask_rcnn_coco_ft_wisdom_color_primesense_20180915-004228.h5'
import h5py
from keras.engine import saving

f = h5py.File(filepath, mode='r')
saving.load_attributes_from_hdf5_group(f, 'layer_names')

In [ ]:
model.load_weights_from_sd_mrcnn_model(filepath='/nfs/diskstation/projects/dex-net/segmentation/models/mask_rcnn_coco_ft_wisdom_color_primesense_20180915-004228.h5')

In [ ]:
model.keras_model.summary()

In [ ]:
results, siamese_output = model.detect([input_image], [target], verbose=1)

In [ ]:
results

In [ ]:
import matplotlib.pyplot as plt
r = results[0]
_, ax = plt.subplots(1,1, figsize=(12,12))
visualize.display_instances(input_image, r['rois'], r['masks'], r['class_ids'], ['bg', 'obj'],
                            r['scores'], ax=ax, title='Predictions')

In [ ]:
siamese_output.shape
for anchor in siamese_output[0]:
    print(np.linalg.norm(anchor, ord=1))